In [32]:
%load_ext autoreload

$$ \newcommand{\ket}[1]{\left|{#1}\right\rangle} $$
$$ \newcommand{\bra}[1]{\left\langle{#1}\right|} $$
$$ \newcommand{\expval}[1]{\left\langle{#1}\right\rangle} $$

# Variational Quantum Eigensolver
BLABLA introduction

### Motivation
Quantum Phase Estimation (QPE) is a common method in quantum computation to estimate eigenvalues of some unitary operator corresponding to an eigenvector. It has been shown to provide an exponential speed-up over current classical algorithms for determining ground state energies of quantum systems. However, to get good accuracy a long coherent evolution has to be performed, and as engineers struggle to achieve acceptable coherence times we can look for other methods better suited for near-term quantum computers. 

The Variational Quantum Eigensolver (VQE), unlike QPE, requires only a short coherent evolution. It is a hybrid algorithm utilizing both quantum and classical computers. In other words, this algorithm is specifically aimed at near-term quantum computers, and only requires state preparation and measurement of Pauli operators which is done in polynomial time. Can implement wave function ansätzes that are exponentially costly on classical computers. VQE is resistant to gate noise in some cases

### Algorithm
Assume we have an initial state of qubits $\ket{\Phi}$, we want to prepare our trial ground-state stationary wave function $\ket{\Psi(\theta)}$, dependent on some set of variables $\theta$, by an action of a suitable unitary operator $\hat{U}(\theta)$.
\begin{equation}
    \ket{\Psi} = \hat{U}\ket{\Phi}
\end{equation}
By variationally optimizing $\hat{U}$ we can find estimates to the ground state energy
\begin{equation}
    \frac{\bra{\Psi}H \ket{\Psi}}{\bra{\Psi}\ket{\Psi}} \leq E_0
\end{equation}
This is commonly known as the variational method. To implement this, we divide our algorithm into two subroutines; one quantum and one classical. The quantum subroutine prepares our state followed by multiple measurements of the expectation value in an appropriate basis. The classical subroutine will optimize the expectation value, with gradient descent or such, by varying the parameters. The process is then repeated.


In [1]:
import numpy as np
import qiskit as qk
from scipy.optimize import minimize

class VariationalQuantumEigensolver:
    def __init__(self,model,method='L-BFGS-B',tol=1e-10,prnt=False,max_iter=1000):
        self.model = model
        self.method = method
        self.tol = tol
        self.prnt = prnt
        self.max_iter = max_iter
        self.theta = self.run()
    
    def ExpectationValue(self,theta, *args):
        model = self.model
        E = 0
        for i in range(model.n):
            for j in range(i+1,model.n):
                qb = qk.QuantumRegister(model.n)
                cb = qk.ClassicalRegister(model.n)
                qc = qk.QuantumCircuit(qb,cb)

                # ansatz
                qc,qb = model.ansatz(qc,qb,theta)

                # pauli word
                qc,qb = model.expval(qc,qb,i,j)

                # measurement
                E += model.measure(qc,qb,cb,i,j)
        if self.prnt:
            print(model.get_eigenstate(theta),E)
        return E
    
    def run(self):
        model = self.model
        thetas = []
        E = []
        for i in range(10):
            theta = 2*np.pi*np.random.randn(model.n)
            thetas.append(theta)
            E.append(self.ExpectationValue(theta,model))
        E = np.array(E)
        print('E: ',E)
        thetas = np.array(thetas)
        arg = np.argmin(E)
        theta = thetas[arg]
        if self.method == 'L-BFGS-B':
            bounds = [(0,2*np.pi) for i in theta]
        else:
            bounds = None
        result = minimize(self.ExpectationValue,theta,bounds=bounds,method=self.method,options={'disp':True,'maxiter':self.max_iter},tol=self.tol)
        expectation = result.x
        print(self.ExpectationValue(expectation))
        return expectation

## Example:
# Heisenberg model
The Heiseinberg model is a one-dimensional chain of spin $\frac{1}{2}$ particles, where there are only one degree of freedom being the spin. The direction of the spins and the relationship between neighbouring spins determine the energy of the system. The Hamiltonian is given by

$$
    H = h \sum_k \sigma_k^z \pm J\sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}}
$$
We write this in terms of dimensionless variables, by dividing with $\pm J$

$$
    \frac{H'}{\pm J} = h' \sum_k \sigma_k^z + \sum_k \vec{\sigma_k} \cdot \vec{\sigma_{k+1}}
$$

with $h'=\frac{h}{\pm J}$. 
Next we write out $\sigma$ vector dot products as

$$
\vec{\sigma_k} \cdot \vec{\sigma_{k+1}} = \sigma_k^x\sigma_{k+1}^x + \sigma_k^y\sigma_{k+1}^y + \sigma_k^z\sigma_{k+1}^z
$$

We are then left with the following expectation values
\begin{align}
    \expval{H'} = h'\sum_k \expval{\sigma_k^z} + \sum_k \left[ \expval{\sigma_k^x\sigma_{k+1}^x} + \expval{\sigma_k^y\sigma_{k+1}^y} + \expval{\sigma_k^z\sigma_{k+1}^z} \right]
\end{align}

In [2]:
class Heisenberg:
    def __init__(self,particles,h):
        self.n = particles
        self.expvals = 4
        self.parameters = [h,1,1,1]
        self.shots = 1024
        
    def ansatz(self,qc,qb,theta):
        for i in range(self.n):
            #qc.ry(theta[i],qb[i])
            qc.h(qb[i])
            qc.rz(theta[i],qb[i])
        for i in range(self.n-1):
            qc.cx(qb[i],qb[i+1])
        qc.cx(qb[-1],qb[0])
        
        return qc,qb
    
    def expval(self,qc,qb,i,k):
        if k == self.n-1 and i != 0:
            k = -1
        if i == 0:
            # Onebody
            qc.z(qb[k])
        
        if i == 1:
            # Twobody X
            qc.x(qb[k])
            qc.x(qb[k+1])
            
        if i == 2:
            # Twobody Y
            qc.y(qb[k])
            qc.y(qb[k+1])
            
        if i == 3:
            # Twobody Z   
            qc.z(qb[k])
            qc.z(qb[k+1])
        return qc,qb
            
    def measure(self,qc,qb,cb,i,k):
        if k == self.n-1 and i != 0:
            k = -1
        
        if i == 1:
            # Twobody X
            qc.h(qb[k])
            qc.h(qb[k+1])
            
        if i == 2:
            # Twobody Y
            qc.h(qb[k])
            qc.h(qb[k+1])
            
        if i == 3:
            # Twobody Z
            qc.cx(qb[k+1],qb[k])
        
        qc.measure(qb,cb)
        job = qk.execute(qc,backend=qk.Aer.get_backend('qasm_simulator'),shots=self.shots)
        res = job.result().get_counts()
        E = 0
        for key,hits in res.items():
            eigenstate = key[::-1]
            if i == 0:
                q1 = 1 if eigenstate[ k ] == '0' else -1
                E += q1*hits
            else:
                q1 = 1 if eigenstate[ k ] == '0' else -1
                q2 = 1 if eigenstate[k+1] == '0' else -1
                eigenvalue = q1*q2
                E += eigenvalue*hits
        E /= self.shots
        return E*self.parameters[i]

    def ExpectationValue(self,theta):
        H = 0
        for k in range(self.n):
            for i in range(self.expvals):
                qb = qk.QuantumRegister(self.n)
                cb = qk.ClassicalRegister(self.n)
                qc = qk.QuantumCircuit(qb,cb)

                # ansatz
                qc,qb = self.ansatz(qc,qb,theta)

                # pauli word
                qc,qb = self.expval(qc,qb,i,k)

                # measurement
                H += self.measure(qc,qb,cb,i,k)

        return H

In [3]:
model = Heisenberg(2,1)
e = VariationalQuantumEigensolver(model)
print(e)

E:  [-0.04882812  0.00585938  0.02539062 -0.0703125   0.         -0.0625
  0.04492188 -0.01367188  0.02929688  0.01367188]
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f= -5.66406D-02    |proj g|=  2.79882D+00

At iterate    1    f= -6.83594D-02    |proj g|=  2.79882D+00

At iterate    2    f=  3.71094D-02    |proj g|=  6.28319D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      2     26      3     0     1   6.283D+00   3.711D-02
  F =   3.7109375000000000E-002

CONVERGE

## Example:
# Maxcut problem
Assume a graph consisting of $n$ nodes with two possible values (e.g. spin). Between some of the nodes there are assigned a value describing the connection. If the two nodes that make up such a connection have different spin you add this value to a sum. The goal of the maxcut problem is to find the arrangement of spins to the graph that maximize this sum.

If we say that the $i$th node can have either the value $x_i=0$ or $x_i=1$ then we can assign this problem a cost function to maximize

\begin{equation}
    C(x) = \sum_{i,j} w_{ij}x_i(1-x_j)
\end{equation}

where $w_ij$ are the matrix element of the matrix describing the graph. As we want to implement this on a quantum computer we know that the pauli-z operator has eigenvalues $z_i \in \{-1,1\}$. We can then make the variable shift
$$ x_i = \frac{1-z_i}{2} $$
Clearly this is holds. We can then write the cost function in terms of pauli-z operators

\begin{align}
    H &= \frac{1}{2}\sum_{i<j} w_{ij}(1-\sigma_{z}^i)(1+\sigma_{z}^j)\\
      &= -\frac{1}{2}\sum_{i<j} w_{ij}\sigma_{z}^i\sigma_{z}^j + const.
\end{align}

which is equivalent to the Ising Hamiltonian without an external field. In the VQE sense the problem is then to estimate the expectation value
\begin{equation}
    \sum_{i<j}\expval{\sigma_{z}^i\sigma_{z}^j}
\end{equation}
and minimize the negative of the Hamiltonian time $-1$.

In [4]:
import time

class Maxcut:
    def __init__(self,nodes,w,shots=1024):
        """
            w -> graph matrix
        """
        self.n = nodes
        self.expvals = 1
        self.w = w
        #self.parameters = 
        self.shots = shots
        self.t1 = -1
        self.t2 = 0
        
    def ansatz(self,qc,qb,theta):
        for i in range(self.n):
            qc.ry(theta[i],qb[i])
        for i in range(self.n-1):
            qc.cx(qb[i],qb[i+1])
        qc.cx(qb[-1],qb[0])
        
        return qc,qb
    
    def expval(self,qc,qb,i,j):
        qc.z(qb[i])
        qc.z(qb[j])
        return qc,qb
            
    def measure(self,qc,qb,cb,i,j):
        #qc.cx(qb[j],qb[i])
        qc.measure(qb,cb)
        job = qk.execute(qc,backend=qk.Aer.get_backend('qasm_simulator'),shots=self.shots)
        res = job.result().get_counts(qc)
        E = 0
        for key,hits in res.items():
            eigenstate = key[::-1]
            q1 = 1 if eigenstate[i] == '0' else -1
            q2 = 1 if eigenstate[j] == '0' else -1
            eigenvalue = q1*q2
            E += eigenvalue*hits
        E /= self.shots
        return 0.5*E*self.w[i,j]

    def get_eigenstate(self,theta):
        qb = qk.QuantumRegister(self.n)
        cb = qk.ClassicalRegister(self.n)
        qc = qk.QuantumCircuit(qb,cb)

        # ansatz
        qc,qb = self.ansatz(qc,qb,theta)
        qc.measure(qb,cb)
        job = qk.execute(qc,backend=qk.Aer.get_backend('qasm_simulator'),shots=self.shots)
        res = job.result().get_counts(qc)
        V = []
        for key,hits in res.items():
            eigenstate = key[::-1]
            V.append([eigenstate,hits])
        return V  

Make a simple graph to test:

    A --- 1 --- B
    |         / |
    |       /   |
    2     3     4
    |  /        |
    |/          |
    D --- 3 --- C
    
represented by the matrix
$$
w = 
\begin{bmatrix}
    0 & 1 & 0 & 2 \\
    1 & 0 & 4 & 3 \\
    0 & 4 & 0 & 3 \\
    2 & 3 & 3 & 0 \\
\end{bmatrix}
$$

And we set up the eigenstates as $\ket{ABCD}$, with the correct eigenstate $\ket{0100}$ or $\ket{1011}$.

In [5]:
w = np.array([[0,1,0,2],[1,0,4,3],[0,4,0,3],[2,3,3,0]])
print(w)
model1 = Maxcut(4,w)
theta = VariationalQuantumEigensolver(model1,method='powell',prnt=False,max_iter=1).theta
V = model1.get_eigenstate(theta)
print(V)

[[0 1 0 2]
 [1 0 4 3]
 [0 4 0 3]
 [2 3 3 0]]
E:  [ 0.04882812 -2.30957031  4.62109375 -0.66992188 -3.4609375  -0.52246094
 -3.078125    3.89941406 -2.046875   -0.02050781]
-3.4970703125
[['0101', 6], ['1101', 2], ['0011', 13], ['0001', 1], ['1010', 1002]]


In [6]:
def maxcut_loss(matrix,vector):
    new = [0 for i in vector]
    for i in range(len(vector)):
        new[i] = vector[i]
        if vector[i] == 0:
            new[i] = -1
    n,m = matrix.shape
    k = 0
    for i in range(n-1):
        for j in range(i+1,m):
            val = matrix[i,j]*new[i]*new[j]
            k += val
    return -0.5*k